# Intelligent Systems - Individual Project Assessment
I aim to split the code into 3 sections:
1. Generating and analysing the datset
2. Developing my classification model
3. Training my model

In [275]:
import matplotlib.pyplot as plt
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import scipy.io as sio
import numpy as np

In [276]:
# set the selected device for the tensors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(torch.cuda.get_device_name(device))
torch.set_default_device(device)

Using device: cuda
NVIDIA GeForce RTX 3050 Laptop GPU


## Dataset

### Data Augmentation
After inspection of the dataset, we have PIL images. Therefore, we will convert these to Tensors.

The values used for the normalisation of data were calculated from the ImageNet training datase

In [277]:
# We perform random transformations to better generalise the training dataset
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(size=(224, 224), antialias=True),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225]) 
])
valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                 std=[0.229, 0.224, 0.225])
])

### Downloading and splitting the dataset

In [278]:
# I will download the data from PyTorch's website and use the appropriate data loader
train_dataset = datasets.Flowers102(
    root='flowers102',
    split="train",
    download=True,
    transform=train_transform
)

valid_dataset = datasets.Flowers102(
    root='flowers102',
    split="val",
    download=True,
    transform=valid_transform
)

test_dataset = datasets.Flowers102(
    root='flowers102',
    split="test",
    download=True,
    transform=test_transform
)

# Get the targets and ids
image_labels = sio.loadmat("flowers102/flowers-102/imagelabels")
setids = sio.loadmat("flowers102/flowers-102/setid")

In [279]:
# look at the first training sample
image, label = train_dataset[0]
print(f"Image shape: {image.shape} -> [batch, height, width]")
print(f"Datatype: {image.type}")
print(f"Label: {image_labels['labels'][label]}")
print(f"Device tensor is stored on: {image.device}")

Image shape: torch.Size([3, 224, 224]) -> [batch, height, width]
Datatype: <built-in method type of Tensor object at 0x000001BD04806670>
Label: [77 77 77 ... 62 62 62]
Device tensor is stored on: cpu


## Model
#### Hyperparameters

### Residual Block
As I wish to implement a model based upon a Residual Network, I must first build a *Residual Block*. A basic block consists of two sequential 3x3 convolutional layers and a residual connection, where the input and output dimensions of the layers are the same.

In [280]:
class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        
        # Create the skip connection len(in_channels) != len(out_channels) or stride != 1
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()
        
    def forward(self, x):
        residual = x.clone()
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        # Skip connection
        residual = self.shortcut(residual)
        x += residual
        x = self.relu(x)
        return x

### ResNet Architecture

In [281]:
class MyNN(nn.Module):
    
    def __init__(self, ResidualBlock, layers, image_channels, num_classes):
        """
        My Neural Network architecture implemented in PyTorch.
        :param ResidualBlock: A residual block class
        :param layers: A list stating how many times we wish to use the residual block for each layer
        :param image_channels: the number of channels of the input image
        :param num_classes: the number of classes
        """
        super(MyNN, self).__init__()
        
        self.in_channels = 64
        # Based upon a residual network, the first layer is a convolutional layer producing 64 channels
        self.conv1 = nn.Conv2d(image_channels, 64, kernel_size=7, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Architecture's layers
        self.layer1 = self.make_layer(ResidualBlock, layers[0], out_channels=64, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, layers[1], out_channels=128, stride=2)
        self.layer3 = self.make_layer(ResidualBlock, layers[2], out_channels=128, stride=2)
        self.layer4 = self.make_layer(ResidualBlock, layers[3], out_channels=512, stride=2)
        
        # Get a single value at the end (scalar)
        self.avgPool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc = nn.Linear(512, num_classes)
    
    def forward(self, x):
        # initial layer setup for the network
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # The architecture's layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Make it the correct dimension
        x = self.avgPool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x
            
    def make_layer(self, ResidualBlock, n_res_blocks, out_channels, stride):
        layers = []
        
        layers.append(ResidualBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels     # no expansion as input channels must equal output channels
        
        for i in range(1, n_res_blocks):
            layers.append(ResidualBlock(self.in_channels, out_channels))
        
        # Return a sequence of modules that is the same as the layers list (including order)  
        return nn.Sequential(*layers)

In [282]:
net = MyNN(ResidualBlock, [2, 2, 2, 2], 3, 1)
net.to(device)
print(net)

MyNN(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (shortcut): Sequential()
    )
    (1): ResidualBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding

In [283]:
# Get the learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size())

79
torch.Size([64, 3, 7, 7])


## Train Network

In [284]:
torch.manual_seed(30)
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [285]:
# optimiser = torch.optim.Adam(net.parameters(), lr=1e-3)

# try to predict the first training sample
image, label = train_dataset[0]
image = image.to(device)
image

tensor([[[ 0.6563,  0.5193,  0.6392,  ..., -1.8097, -1.0733, -0.6452],
         [ 0.6734,  0.5707,  0.7077,  ..., -1.8268, -1.0733, -0.6794],
         [ 0.7419,  0.6734,  0.7591,  ..., -1.7925, -1.0390, -0.7308],
         ...,
         [ 0.5022,  0.5193,  0.4508,  ..., -0.0972,  0.5536,  0.8276],
         [ 0.5193,  0.5364,  0.4166,  ..., -0.1486,  0.5707,  0.8276],
         [ 0.5193,  0.4851,  0.4166,  ..., -0.1143,  0.5878,  0.8961]],

        [[ 0.3803,  0.2402,  0.3627,  ..., -1.5980, -0.7577, -0.2850],
         [ 0.3978,  0.2752,  0.3978,  ..., -1.5805, -0.7402, -0.3200],
         [ 0.4503,  0.3627,  0.4503,  ..., -1.5105, -0.7052, -0.3375],
         ...,
         [ 0.0126,  0.0301, -0.0224,  ..., -0.0049,  0.6078,  0.9755],
         [ 0.0301,  0.0476, -0.0224,  ..., -0.0049,  0.6779,  0.9230],
         [ 0.0651,  0.0476, -0.0049,  ...,  0.0476,  0.7304,  0.9580]],

        [[ 1.2108,  1.0539,  1.1759,  ..., -1.5953, -0.9504, -0.6193],
         [ 1.2282,  1.0888,  1.2282,  ..., -1

In [286]:
print(image.unsqueeze(0).shape)

torch.Size([1, 3, 224, 224])


In [287]:
prediction = net(image.unsqueeze(0))
prediction.shape

torch.Size([1, 1])

In [288]:
# Compare to label
print(f"Target label: {label}")
print(f"Prediction: {prediction}")

Target label: 0
Prediction: tensor([[1.6499]], device='cuda:0', grad_fn=<AddmmBackward0>)
